In [13]:
#создание матрицы процентного содержания материалов data

import pandas as pd
import numpy as np
#np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt
#import collections as col

pdRec = pd.read_csv('Recipes_p.csv', header = 0)

#pdRec.sort_values(['HEATID'],ascending=True,axis=0,inplace=True,na_position='last')
#pdRec.to_csv(r'Recipes_sort.csv',index=None,header=True)

#HEATID,Material_Code,NAME,WEIGHT
rec=pdRec.to_numpy()
order = rec[:, 1].argsort()
rec = np.take(rec, order, 0)#сортируем по номеру материала
i=0
temp=rec[0,1]
for a in rec:
    if (a[1]!=temp):
        i+=1
        temp=a[1]
i+=1
print("материалов всего ",i)
nmat=i
mat_code=np.zeros(nmat)#вектор с идентификаторами материалов

i=0
temp=rec[0,1]
mat_code[0]=temp
for a in rec:
    if (a[1]!=temp):
        i+=1
        temp=a[1]
        mat_code[i]=temp
i+=1
print("Коды материалов: ",mat_code)

rec=pdRec.to_numpy()#отсортированная по heat_id 

def writ(i,a):#записывает проценты в нужный столбец строки i
    hdata[i]=a[0]
    for n in range(mat_code.shape[0]):
        if(mat_code[n]==a[1]):
            data[i,n]=a[4]
            break
    return()

#подсчет head
i=0
temp=rec[0,0]
for a in rec:
    if (a[0]!=temp):
        i+=1
        temp=a[0]
i+=1
data=np.zeros((i,mat_code.shape[0]))
print("Строк в массиве data: ",i)

hdata=np.zeros(i,dtype=int)#heat_data for data

i=0
temp=rec[0,0]
for a in rec:
    if (a[0]!=temp):
        i+=1
        temp=a[0]
    writ(i,a)
    
pdData=pd.DataFrame(data,index=hdata.reshape(1,len(hdata)).tolist(),columns=mat_code.tolist())
pdData.to_csv(r'DataR.csv',index=True,header=True)
pdData.head(100)

print('hdata: ', hdata, hdata.shape)

pdh=pd.DataFrame(hdata,index=None)
pdh.to_csv(r'HEAT_ID.csv',index=None,header=False)


материалов всего  29
Коды материалов:  [112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 124. 125. 126.
 166. 180. 181. 182. 183. 190. 207. 208. 209. 210. 211. 212. 213. 214.
 215.]
Строк в массиве data:  12746
hdata:  [1701190 1701192 1701194 ... 1811202 1811204 1811206] (12746,)


In [15]:
#создание матрицы векторов с дефектами def_data

import pandas as pd
import numpy as np
#np.set_printoptions(threshold=np.inf)
import matplotlib.pyplot as plt

pdTDef=pd.read_csv('Defect_correlation.csv', header = 0,sep=";")
pdDefect = pd.read_csv('Defects_sort.csv', header = 0)
pdDefect.sort_values(['HEAT_ID'],ascending=True,axis=0,inplace=True,na_position='last')

pdhdata=pd.read_csv('HEAT_ID.csv',header=None)
#DEFECT_CODE=DEFECT_TYPE
#DEFECT_CODE;DEFECT_TEXT_EN;DEFECT_GROUP_CODE;DEFECT_GROUP_TEXT_EN;FROM_RAW_MATERIAL
TDef=pdTDef.to_numpy()

#PIECE_ID,HEAT_ID,DEFECT_TYPE,DEFECT_NAME,DEFECT_GROUP_ID,DEFECT_GROUP_Name
Defect=pdDefect.to_numpy()

hdata=pdhdata.to_numpy()

#print(hdata.reshape(1,len(hdata)).tolist())

#вектор номеров дефектов, зависящих от материала defect_type
defect_type=list()

for a in TDef:
    if(a[4]=="TRUE")or(a[4]=="MAYBE"):
        if ((a[0] in defect_type)==False):
            defect_type.append(a[0])
        else:
            print("Дубликат дефекта!")
defect_type.sort()
print(defect_type)

#матрица с дефектами
DataD=np.zeros((hdata.shape[0],len(defect_type)),dtype=int)

print(DataD.shape)

#пишем в Defect2 только те строки, где есть наш дефект из defect_type
#В итоге, ячейки, которые с дефектом получают "+1"(при каждом повторе head)
#Если нет информации о дефекте в Recipes, то он поподает в черный список Black

black=list()
for z in range(len(defect_type)):
    i=0
    dt=defect_type[z]
    for d in Defect:
        if(d[2]==dt):
            i+=1
    print("Новое кол-во сторок в defect: ", i)
    Defect2=np.empty((i,Defect.shape[1]),dtype=object)   
    i=0
    for d in Defect:
        if(d[2]==dt):
            Defect2[i]=d
            i+=1

    print(Defect2)
    #заполняем матрицу дефектов
    j=0
    temp=0
    for d in Defect2:
        j=temp
        while (j < hdata.shape[0]):
                if(d[1]==hdata[j]):
                    print(j)
                    print(d)
                    DataD[j,z]=1#DataD[j,z]+=1
                    print(DataD[j,z])
                    temp=j
                    break
                if(j==hdata.shape[0]-1):   
                    print("Для этой плавки у нас нет информации:",d)
                    print("Дефект ",defect_type[z])
                    for x in d:
                        black.append(x)
                j+=1
                    
print(DataD)
Ab=np.array(black).reshape(len(black)//Defect2.shape[1],Defect2.shape[1])

pdDataD=pd.DataFrame(DataD,index=hdata.reshape(1,len(hdata)).tolist(),columns=defect_type)
pdDataD.to_csv(r'DataD.csv',index=True,header=True)
pdBlack=pd.DataFrame(Ab,index=None)
pdBlack.to_csv(r'Black.csv',index=None,header=False)

DataPart=np.zeros((hdata.shape[0],len(defect_type)),dtype=int)



[48, 73, 131, 132, 133, 134, 135, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 170, 171, 172, 196, 211, 217, 230]
(12746, 26)
Новое кол-во сторок в defect:  7
[[292970 1713830 48 'Hot Shortness' 'SU' '1-Surface']
 [292969 1713830 48 'Hot Shortness' 'SU' '1-Surface']
 [292968 1713830 48 'Hot Shortness' 'SU' '1-Surface']
 [292971 1713830 48 'Hot Shortness' 'SU' '1-Surface']
 [292973 1713830 48 'Hot Shortness' 'SU' '1-Surface']
 [292972 1713830 48 'Hot Shortness' 'SU' '1-Surface']
 [292974 1713830 48 'Hot Shortness' 'SU' '1-Surface']]
5523
[292970 1713830 48 'Hot Shortness' 'SU' '1-Surface']
1
5523
[292969 1713830 48 'Hot Shortness' 'SU' '1-Surface']
1
5523
[292968 1713830 48 'Hot Shortness' 'SU' '1-Surface']
1
5523
[292971 1713830 48 'Hot Shortness' 'SU' '1-Surface']
1
5523
[292973 1713830 48 'Hot Shortness' 'SU' '1-Surface']
1
5523
[292972 1713830 48 'Hot Shortness' 'SU' '1-Surface']
1
5523
[292974 1713830 48 'Hot Shortness' 'SU' '1-Surface']
1
Новое кол-во сторок в defec

Для этой плавки у нас нет информации: [215934 1705552 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
Дефект  133
Для этой плавки у нас нет информации: [215935 1705552 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
Дефект  133
Для этой плавки у нас нет информации: [215936 1705552 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
Дефект  133
Для этой плавки у нас нет информации: [215937 1705552 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
Дефект  133
Для этой плавки у нас нет информации: [215938 1705552 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
Дефект  133
2382
[230591 1706938 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
1
2675
[234530 1707590 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
1
2675
[234533 1707590 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
1
2675
[234534 1707590 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
1
3123
[245869 1708566 133 'Non-Metallic inclusions' 'MET' '9-Metallurgy']
1
3671
[253668 1709828 133 'Non-Metal

Новое кол-во сторок в defect:  4540
[[167951 1701216 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
 [167956 1701216 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
 [167952 1701216 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
 ...
 [462713 1811154 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
 [462714 1811154 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
 [462715 1811154 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']]
Для этой плавки у нас нет информации: [167951 1701216 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [167956 1701216 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [167952 1701216 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [167953 1701216 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [167954 1701216 134 'Off Chemistry BRS Chem'

Для этой плавки у нас нет информации: [168969 1701262 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168973 1701262 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168145 1701266 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168149 1701268 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168150 1701268 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168151 1701268 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168152 1701268 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168153 1701268 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [168154 1701268 134 'Off C

Для этой плавки у нас нет информации: [173533 1701698 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [173535 1701698 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [173534 1701698 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [173536 1701698 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [173543 1701700 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [173541 1701700 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
Для этой плавки у нас нет информации: [173542 1701700 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
Дефект  134
167
[174336 1701746 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
1
168
[174346 1701748 134 'Off Chemistry BRS Chem' 'MET' '9-Metallurgy']
1
168
[174344 1701748 134 'Off Chemis

...

10395
[395779 1806278 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10396
[396039 1806280 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10396
[396042 1806280 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10396
[396041 1806280 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10396
[396040 1806280 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10396
[396038 1806280 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10396
[396043 1806280 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10396
[396037 1806280 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[395788 1806282 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[395789 1806282 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[395790 1806282 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[395792 1806282 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[397491 1806282 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[395791 1806282 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[395793 1806282 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10397
[395

[398625 1806548 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10522
[398628 1806548 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10522
[398627 1806548 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10522
[398626 1806548 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10522
[399304 1806548 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10522
[398629 1806548 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10522
[399303 1806548 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10523
[398631 1806550 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10523
[398630 1806550 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10523
[398633 1806550 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10523
[398632 1806550 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10523
[398635 1806550 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
10523
[398634 1806550 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
Для этой плавки у нас нет информации: [398637 1806552 211 'Off Chemistry' 'MET' '9-Metallurgy']
Дефект  211
Для этой плавки у нас нет информации: [3

[446654 1809852 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12095
[446658 1809852 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12095
[446655 1809852 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12095
[446656 1809852 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12095
[446659 1809852 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12095
[446657 1809852 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12096
[446663 1809854 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12096
[446662 1809854 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12096
[446661 1809854 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12096
[446665 1809854 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12096
[446666 1809854 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12096
[446660 1809854 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12096
[446664 1809854 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12097
[446668 1809856 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12097
[446667 1809856 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12098
[446677 18

12443
[452889 1810586 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12443
[452895 1810586 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12443
[452896 1810586 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12444
[452902 1810588 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12444
[452900 1810588 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12444
[452897 1810588 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12444
[452901 1810588 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12444
[452898 1810588 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12444
[452899 1810588 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12445
[452904 1810590 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12445
[452903 1810590 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12445
[452909 1810590 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12446
[452915 1810592 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12446
[452910 1810592 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12446
[452914 1810592 211 'Off Chemistry' 'MET' '9-Metallurgy']
1
12446
[452

Новое кол-во сторок в defect:  2
[[356274 1802772 217 'Cracks: Transverse' 'SU' '1-Surface']
 [362489 1803302 217 'Cracks: Transverse' 'SU' '1-Surface']]
8720
[356274 1802772 217 'Cracks: Transverse' 'SU' '1-Surface']
1
8973
[362489 1803302 217 'Cracks: Transverse' 'SU' '1-Surface']
1
Новое кол-во сторок в defect:  0
[]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
